In [ ]:
!python -m spacy download pt

In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import plotly.graph_objects as go
import nltk
import spacy

In [2]:
# Leitura dos csv
df_course = pd.read_csv('data/course.csv')
df_offers = pd.read_csv('data/offers.csv')

In [3]:
spacy_pt = spacy.load('pt_core_news_sm')
stop_words = set(stopwords.words('portuguese') + list(punctuation))

In [4]:
# Funcoes para processar os textos
def fillNADropDuplicated(df, columns_check):
    df_clean = df.copy()
    #Arrumando o nome das colunas
    df_clean.columns = [col.replace('”','').replace('“','') for col in df_clean.columns]
    
    #Removendo os registros com as colunas especificadas duplicadas, mantendo o primeiro registro
    df_clean = df_clean.drop_duplicates(subset =columns_check,keep = "first")
    
    #Verificando se a quantidade de valores NA é menor que 5%, se for, é deletado os registros, 
    #se não, é feito um fill de string vazia
    total = len(df_clean)
    for col in df_clean.columns:
        col_na = df_clean[col].isna().sum()
        col_na_perc = round(col_na/total*100,2)
        
        if col_na_perc < 5.00:
            df_clean = df_clean[df_clean[col].notna()]
            
        else:
            df_clean[col].fillna('', inplace=True)
            
    return df_clean

    

def preProcessingText(text):
    #Transformando o texto tudo em minusculo, removendo stopwords e pontuações
    clean_text = ' '.join([desc.lower() for desc in text.split() if desc not in stop_words])
    #Lemmatização dos verbos
    processed_text = spacy_pt(clean_text)
    clean_text = ' '.join([token.lemma_ if token.pos_ == 'VERB' else token.text for token in processed_text])
    
    return clean_text


In [6]:
df_course_clean = fillNADropDuplicated(df_course,["DISCIPLINA","CONTEUDO"])
df_course_clean

,ID,DISCIPLINA,CONTEUDO
0,1,C1,Apresentação da componente curricular ementas-...
2,3,C1,Apresentação dos principais conceitos da lógic...
4,5,C1,Fluxogramas- introdução a psudolinguagem ativi...
8,9,C1,Conceitos básicos para a construção de algorit...
12,13,C1,Descrição do funcionamento das estruturas de r...
...,...,...,...
2353,2354,C3,Atividades práticas.
2355,2356,C3,Avaliação teórico - prática
2359,2360,C3,Término da avaliação prática
2363,2364,C3,Introdução ao Maven; exemplos práticos.


In [7]:
# Criando a coluna do conteudo clean (processado)
df_course_clean['CONTEUDO_CLEAN'] = df_course_clean['CONTEUDO'].apply(lambda x: preProcessingText(x))
df_course_clean

,ID,DISCIPLINA,CONTEUDO,CONTEUDO_CLEAN
0,1,C1,Apresentação da componente curricular ementas-...,apresentação componente curricular ementas- bi...
2,3,C1,Apresentação dos principais conceitos da lógic...,apresentação principais conceitos lógica progr...
4,5,C1,Fluxogramas- introdução a psudolinguagem ativi...,fluxogramas- introdução psudolinguagem ativida...
8,9,C1,Conceitos básicos para a construção de algorit...,conceitos básicos construção algoritmos comput...
12,13,C1,Descrição do funcionamento das estruturas de r...,descrição funcionamento estruturas repetição ....
...,...,...,...,...
2353,2354,C3,Atividades práticas.,atividades práticas .
2355,2356,C3,Avaliação teórico - prática,avaliação teórico prática
2359,2360,C3,Término da avaliação prática,término avaliação prática
2363,2364,C3,Introdução ao Maven; exemplos práticos.,introdução maven ; exemplos práticos .
